In [1]:
# uncomment the next line if you want to delete an existing models directory
!rm -rf ./models/
# clone the Tensorflow Model Garden
!git clone --depth 1 https://github.com/tensorflow/models/

# install the Object Detection API
!cd models/research/ && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

Cloning into 'models'...
remote: Enumerating objects: 4108, done.
remote: Counting objects: 100% (4108/4108), done.
remote: Compressing objects: 100% (3109/3109), done.
remote: Total 4108 (delta 1188), reused 2037 (delta 937), pack-reused 0
Receiving objects: 100% (4108/4108), 45.36 MiB | 49.52 MiB/s, done.
Resolving deltas: 100% (1188/1188), done.
Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Using cached tensorflow-2.16.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (590.6 MB)
  Using cached ml_dtypes-0.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.2 MB)
  Using cached tensorboard-2.16.2-py3-none-any.whl (5.5 MB)
  Using cached keras-3.4.1-py3-none-any.whl (1.1 MB)
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1697356 sha256=7954212c61a9f08122f18d942e3f92650a970d8c26e6003783faea4ec02f4fe8
  Stored in directory: /tmp/pip-ephem-wheel-cache-l2jmj7d1/wheels/53/dd/70/2de274d6c443

In [2]:
# need to use an older version of tensorflow
!pip install tensorflow==2.15.0

  Using cached tensorflow-2.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (475.2 MB)
  Using cached ml_dtypes-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached tensorboard-2.15.2-py3-none-any.whl (5.5 MB)
  Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.2
    Uninstalling tensorflow-2.16.2:
      Successfully uninstalled tensorflow-2.16.2
ERROR: pip's dependency re

In [4]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import zipfile
import io
import scipy.misc
import numpy as np

import glob
import imageio
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage
import cv2

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [5]:
print(tf.__version__)

2.15.0


In [6]:
%cd models/research/object_detection/
# import the label map utility module
from utils import label_map_util

# import module for reading and updating configuration files.
from utils import config_util

# import module for visualization. use the alias `viz_utils`
from utils import visualization_utils as viz_utils

# import module for building the detection model
from builders import model_builder
### END CODE HERE ###

# import module for utilities in Colab
from object_detection.utils import colab_utils

/content/models/research/object_detection


In [ ]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
    path: a file path.

    Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
    """

    # img_data = tf.io.gfile.GFile(path, 'rb').read()
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))

    image_np = np.array(image)
    image_np_3d = np.repeat(image_np[:, :, np.newaxis], 3, axis=2)

    (im_width, im_height) = image.size
    return image_np_3d.reshape(
        (im_height, im_width, 3)).astype(np.uint8)
    # return np.array(image.getdata()).reshape(
    #     (im_height, im_width)).astype(np.uint8)


def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
    """Wrapper function to visualize detections.

    Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
          and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
          this function assumes that the boxes to be plotted are groundtruth
          boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
          category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
    """

    image_np_with_annotations = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_annotations,
        boxes,
        classes,
        scores,
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=500, # 20
        min_score_thresh=0.3) # 0.8

    if image_name:
        plt.imsave(image_name, image_np_with_annotations)

    else:
        plt.imshow(image_np_with_annotations)

**Load and Visualize Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# image = cv2.imread('/content/drive/MyDrive/Cell_Mem_Dataset/PNGImages/1.png')
train_image_dir = '/content/drive/MyDrive/Database/training_calcium_assay_jpg/6_clear_imgs'
# declare an empty list
train_images_np = []

# run a for loop for each image, (1, Num_img+1)
for i in range(1, 7):

    # define the path (string) for each image
    image_path = os.path.join(train_image_dir+'/'+str(i)+'.jpg')
    print(image_path)

    # load images into numpy arrays and append to a list
    train_images_np.append(load_image_into_numpy_array(image_path))
### END CODE HERE ###

# configure plot settings via rcParams
plt.rcParams['axes.grid'] = False
plt.rcParams['xtick.labelsize'] = False
plt.rcParams['ytick.labelsize'] = False
plt.rcParams['xtick.top'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False
plt.rcParams['ytick.right'] = False
plt.rcParams['figure.figsize'] = [14, 7]

# plot images
for idx, train_image_np in enumerate(train_images_np):
    plt.subplot(1, 6, idx+1)
    plt.imshow(train_image_np)

plt.show()

**Prepare Data for training**

Draw my own ground truth box

In [ ]:
# Define the list of ground truth boxes
gt_boxes = []
# Option 1: draw your own ground truth boxes

# annotate the training images
colab_utils.annotate(train_images_np, box_storage_pointer=gt_boxes)

# Option 1: draw your own ground truth boxes
# TEST CODE:
try:
  assert(len(gt_boxes) == 5), "Warning: gt_boxes is empty. Did you click `submit`?"

except AssertionError as e:
  print(e)

# checks if there are boxes for all 5 images
for gt_box in gt_boxes:
    try:
      assert(gt_box is not None), "There are less than 5 sets of box coordinates. " \
                                  "Please re-run the cell above to draw the boxes again.\n" \
                                  "Alternatively, you can run the next cell to load pre-determined " \
                                  "ground truth boxes."

    except AssertionError as e:
        print(e)
        break


# ref_gt_boxes = [
#         np.array([[0.27333333, 0.41500586, 0.74333333, 0.57678781]]),
#         np.array([[0.29833333, 0.45955451, 0.75666667, 0.61078546]]),
#         np.array([[0.40833333, 0.18288394, 0.945, 0.34818288]]),
#         np.array([[0.16166667, 0.61899179, 0.8, 0.91910903]]),
#         np.array([[0.28833333, 0.12543962, 0.835, 0.35052755]]),
#       ]

# for gt_box, ref_gt_box in zip(gt_boxes, ref_gt_boxes):
#     try:
#       assert(np.allclose(gt_box, ref_gt_box, atol=0.04)), "One of the boxes is too big or too small. " \
#                                                           "Please re-draw and make the box tighter around the cell."

#     except AssertionError as e:
#       print(e)
#       break

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'--boxes array populated--'

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'--boxes array populated--'

<IPython.core.display.Javascript object>

In [ ]:
# print and save the coordinates of your ground truth boxes
from numpy import savetxt
i = 1
for gt_box in gt_boxes:
  print(gt_box)
  # save to csv file
  save_path = os.path.join(train_image_dir+'/'+'boxes_all'+'/'+str(i)+'.csv')
  savetxt(save_path, gt_box, delimiter=' ')
  i += 1

In [ ]:
type(gt_box)

numpy.ndarray

Load Given ground truth boxes

In [ ]:
from numpy import loadtxt
# load saved boxes
gt_boxes = []
for i in range(1,7):
  save_path = os.path.join(train_image_dir+'/'+'boxes_all'+'/'+str(i)+'.csv')
  # gt_box = np.array(loadtxt(save_path, delimiter=' '))
  gt_box = loadtxt(save_path, delimiter=' ')
  gt_boxes.append(gt_box)
  i += 1

  print(gt_boxes)

**Define category index dictionary**

In [ ]:
# Assign the cell class ID
cell_class_id = 1

# define a dictionary describing the cell class
category_index = {cell_class_id :
                 {'id'  : cell_class_id,
                  'name': 'cell'}
                 }

# Specify the number of classes that the model will predict
num_classes = 1

# TEST CODE:

print(category_index[cell_class_id])

{'id': 1, 'name': 'cell'}


**Data preprocessing**

Convert the class labels to one-hot representations
convert everything (i.e. train images, gt boxes and class labels) to tensors.

In [ ]:
# The `label_id_offset` here shifts all classes by a certain number of indices;
# we do this here so that the model receives one-hot labels where non-background
# classes start counting at the zeroth index.  This is ordinarily just handled
# automatically in our training binaries, but we need to reproduce it here.

label_id_offset = 1
train_image_tensors = []

# lists containing the one-hot encoded classes and ground truth boxes
gt_classes_one_hot_tensors = []
gt_box_tensors = []

for (train_image_np, gt_box_np) in zip(train_images_np, gt_boxes):

    # convert training image to tensor, add batch dimension, and add to list
    train_image_tensors.append(tf.expand_dims(tf.convert_to_tensor(
        train_image_np, dtype=tf.float32), axis=0))

    # convert numpy array to tensor, then add to list
    gt_box_tensors.append(tf.convert_to_tensor(gt_box_np, dtype=tf.float32))

    # apply offset to to have zero-indexed ground truth classes
    zero_indexed_groundtruth_classes = tf.convert_to_tensor(
        np.ones(shape=[gt_box_np.shape[0]], dtype=np.int32) - label_id_offset)

    # do one-hot encoding to ground truth classes
    gt_classes_one_hot_tensors.append(tf.one_hot(
        zero_indexed_groundtruth_classes, num_classes))

print('Done prepping data.')

Done prepping data.


In [ ]:
# give boxes a score of 100%
dummy_scores = np.array([1.0], dtype=np.float32)

# define the figure size
plt.figure(figsize=(30, 15))

# use the `plot_detections()` utility function to draw the ground truth boxes
for idx in range(5):
    plt.subplot(2, 4, idx+1)
    plot_detections(
      train_images_np[idx],
      gt_boxes[idx],
      # np.ones(shape=[gt_boxes[idx].shape[0]], dtype=np.int32),
      # dummy_scores, category_index)
      np.ones(shape=[gt_boxes[idx].shape[0]], dtype=np.int32),
      None, category_index)

plt.show()

**Download the checkpoint containing the pre-trained weights**

In [ ]:
### START CODE HERE ###
# Download the SSD Resnet 50 version 1, 640x640 checkpoint
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!mv ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint test_data/

# untar (decompress) the tar file
None

# copy the checkpoint to the test_data folder models/research/object_detection/test_data/
None

### END CODE HERE

**Configure the Model**

In [ ]:
tf.keras.backend.clear_session()

from utils.config_util import get_configs_from_pipeline_file
# define the path to the .config file for ssd resnet 50 v1 640x640
pipeline_config = "/content/models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config"
# Load the configuration file into a dictionary
configs = get_configs_from_pipeline_file(pipeline_config, config_override=None)
# See what configs looks like
configs

get model configuration

In [ ]:
# Read in the object stored at the key 'model' of the configs dictionary
model_config = configs['model']

# see what model_config looks like
model_config

  Modify model_config

In [ ]:
# Modify the number of classes from its default of 90
model_config.ssd.num_classes = num_classes
# Freeze batch normalization
model_config.ssd.freeze_batchnorm = True

model_config.ssd.post_processing.batch_non_max_suppression.max_detections_per_class = 1000
model_config.ssd.post_processing.batch_non_max_suppression.max_total_detections = 1000
model_config.ssd.post_processing.batch_non_max_suppression.iou_threshold = 0.3

# model_config.ssd.box_coder.faster_rcnn_box_coder.y_scale = 3
# model_config.ssd.box_coder.faster_rcnn_box_coder.x_scale = 3
# model_config.ssd.box_coder.faster_rcnn_box_coder.height_scale = 2
# model_config.ssd.box_coder.faster_rcnn_box_coder.height_scale = 2


# See what model_config now looks like after you've customized it!
model_config

Modify train_config

In [ ]:
train_config = configs['train_config']
train_config.data_augmentation_options[1].random_crop_image.max_aspect_ratio = 1.5
train_config.max_number_of_boxes = 1000
train_config

In [ ]:
configs

**Build the model**

Build the custom model

In [ ]:
detection_model = model_builder.build(
      model_config=model_config, is_training=True)

print(type(detection_model))

**Inspect the detection_model**

In [ ]:
# Run this to check the type of detection_model
detection_model

In [ ]:
vars(detection_model)

In [ ]:
# view the type of _box_predictor
detection_model._box_predictor

In [ ]:
vars(detection_model._box_predictor)

**Define Checkpoints for the box predictor**

In [ ]:
tmp_box_predictor_checkpoint = tf.compat.v2.train.Checkpoint(
    _base_tower_layers_for_heads=detection_model._box_predictor._base_tower_layers_for_heads,
    _box_prediction_head=detection_model._box_predictor._box_prediction_head,
    )

# Check the datatype of this checkpoint
type(tmp_box_predictor_checkpoint)

# Check the variables of this checkpoint
vars(tmp_box_predictor_checkpoint)

**Define the temporary model checkpoint**

In [ ]:
tmp_model_checkpoint = tf.compat.v2.train.Checkpoint(
          _feature_extractor=detection_model._feature_extractor,
          _box_predictor=tmp_box_predictor_checkpoint)

tmp_model_checkpoint = tf.compat.v2.train.Checkpoint(model=tmp_model_checkpoint)

# Check the datatype of this checkpoint
type(tmp_model_checkpoint)

# Check the vars of this checkpoint
vars(tmp_model_checkpoint)

**Restore the checkpoint**

In [ ]:
checkpoint_path = '/content/models/research/object_detection/test_data/checkpoint/ckpt-0'

# Define a checkpoint that sets `model= None
checkpoint =tf.train.Checkpoint(model=tmp_model_checkpoint)

# Restore the checkpoint to the checkpoint path
tmp_model_checkpoint.restore(checkpoint_path).expect_partial()

**Run a dummy image to generate the model variables**

In [ ]:
# use the detection model's `preprocess()` method and pass a dummy image
tmp_image, tmp_shapes =  detection_model.preprocess(tf.zeros([1, 968, 548, 3]))

# run a prediction with the preprocessed image and shapes
tmp_prediction_dict = detection_model.predict(tmp_image, tmp_shapes)

# postprocess the predictions into final detections
tmp_detections = detection_model.postprocess(tmp_prediction_dict, tmp_shapes)
print('Weights restored!')

In [ ]:
# Test Code:
assert len(detection_model.trainable_variables) > 0, "Please pass in a dummy image to create the trainable variables."

print(detection_model.weights[0].shape)
print(detection_model.weights[231].shape)
print(detection_model.weights[462].shape)

**Eager mode custom training loop**

**Set training hyperparameters**

In [ ]:
tf.keras.backend.set_learning_phase(True)

# set the batch_size = Num_img-1
batch_size =5

# set the number of batches
num_batches = 100

# Set the learning rate
learning_rate = 0.01

# set the optimizer and pass in the learning_rate
optimizer = tf.keras.optimizers.SGD(
    learning_rate=learning_rate, momentum=0.9)

**Choose the layers to fine-tune**

In [ ]:
for i,v in enumerate(detection_model.trainable_variables):
    print(f"i: {i} \t name: {v.name} \t shape:{v.shape} \t dtype={v.dtype}")

**Select the prediction layer variables**

In [ ]:
# define a list that contains the layers that you wish to fine tune
to_fine_tune = []
for v in detection_model.trainable_variables:
  if v.name.startswith('WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead') or v.name.startswith('WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead') :
    to_fine_tune.append(v)

In [ ]:
# Test Code:

print(to_fine_tune[0].name)
print(to_fine_tune[2].name)

**Train your model**

In [ ]:
# Get a batch of your training images
g_images_list = train_image_tensors[0:2]

In [ ]:
print(g_images_list[0])

In [ ]:
# Use .preprocess to preprocess an image
g_preprocessed_image = detection_model.preprocess(g_images_list[0])
print(f"g_preprocessed_image type: {type(g_preprocessed_image)}")
print(f"g_preprocessed_image length: {len(g_preprocessed_image)}")
print(f"index 0 has the preprocessed image of shape {g_preprocessed_image[0].shape}")
print(f"index 1 has information about the image's true shape excluding padding: {g_preprocessed_image[1]}")

In [ ]:
preprocessed_image_list = []
true_shape_list = []

for img in g_images_list:
    processed_img, true_shape = detection_model.preprocess(img)
    preprocessed_image_list.append(processed_img)
    true_shape_list.append(true_shape)

print(f"preprocessed_image_list is of type {type(preprocessed_image_list)}")
print(f"preprocessed_image_list has length {len(preprocessed_image_list)}")
print()
print(f"true_shape_list is of type {type(true_shape_list)}")
print(f"true_shape_list has length {len(true_shape_list)}")

**Make a prediction**

In [ ]:
# Try to call `predict` and pass in lists; look at the error message
try:
    detection_model.predict(preprocessed_image_list, true_shape_list)
except AttributeError as e:
    print("Error message:", e)

In [ ]:
# Turn a list of tensors into a tensor
preprocessed_image_tensor = tf.concat(preprocessed_image_list, axis=0)
true_shape_tensor = tf.concat(true_shape_list, axis=0)

print(f"preprocessed_image_tensor shape: {preprocessed_image_tensor.shape}")
print(f"true_shape_tensor shape: {true_shape_tensor.shape}")

In [ ]:
# Make predictions on the images
prediction_dict = detection_model.predict(preprocessed_image_tensor, true_shape_tensor)

print("keys in prediction_dict:")
for key in prediction_dict.keys():
    print(key)

 **Calculate Loss**

In [ ]:
try:
    losses_dict = detection_model.loss(prediction_dict, true_shape_tensor)
except RuntimeError as e:
    print(e)

Groundtruth tensor boxes has not been provided


In [ ]:
# Get the ground truth bounding boxes
gt_boxes_list = gt_box_tensors[0:2]

# Get the ground truth class labels
gt_classes_list = gt_classes_one_hot_tensors[0:2]

# Provide the ground truth to the model
detection_model.provide_groundtruth(
            groundtruth_boxes_list=gt_boxes_list,
            groundtruth_classes_list=gt_classes_list)

In [ ]:
# Calculate the loss after you've provided the ground truth
losses_dict = detection_model.loss(prediction_dict, true_shape_tensor)

# View the loss dictionary
losses_dict = detection_model.loss(prediction_dict, true_shape_tensor)
print(f"loss dictionary keys: {losses_dict.keys()}")
print(f"localization loss {losses_dict['Loss/localization_loss']:.8f}")
print(f"classification loss {losses_dict['Loss/classification_loss']:.8f}")

In [ ]:
# Let's just reset the model so that you can practice setting it up yourself!
detection_model.provide_groundtruth(groundtruth_boxes_list=[], groundtruth_classes_list=[])

**Define the training step**

In [ ]:
# decorate with @tf.function for faster training (remember, graph mode!)
@tf.function
def train_step_fn(image_list,
                groundtruth_boxes_list,
                groundtruth_classes_list,
                model,
                optimizer,
                vars_to_fine_tune):
    """A single training iteration.

    Args:
      image_list: A list of [1, height, width, 3] Tensor of type tf.float32.
        Note that the height and width can vary across images, as they are
        reshaped within this function to be 640x640.
      groundtruth_boxes_list: A list of Tensors of shape [N_i, 4] with type
        tf.float32 representing groundtruth boxes for each image in the batch.
      groundtruth_classes_list: A list of Tensors of shape [N_i, num_classes]
        with type tf.float32 representing groundtruth boxes for each image in
        the batch.

    Returns:
      A scalar tensor representing the total loss for the input batch.
    """

    with tf.GradientTape() as tape:
    ### START CODE HERE (Replace instances of `None` with your code) ###

        # Preprocess the images
        preprocessed_image_list=[]
        true_shape_list=[]
        model.provide_groundtruth(
        groundtruth_boxes_list=groundtruth_boxes_list,
        groundtruth_classes_list=groundtruth_classes_list)

        for img in image_list:
          processed_img, true_shape = model.preprocess(img)
          preprocessed_image_list.append(processed_img)
          true_shape_list.append(true_shape)




        preprocessed_image_tensor = tf.concat(preprocessed_image_list, axis=0)
        true_shape_tensor = tf.concat(true_shape_list, axis=0)

        # Make a prediction
        prediction_dict = model.predict(preprocessed_image_tensor, true_shape_tensor)

        # Calculate the total loss (sum of both losses)






        losses_dict = model.loss(prediction_dict, true_shape_tensor)
        total_loss=losses_dict['Loss/localization_loss']+losses_dict['Loss/classification_loss']
        # Calculate the gradients
        gradients = tape.gradient(total_loss,vars_to_fine_tune)
        optimizer.apply_gradients(zip(gradients, vars_to_fine_tune))
        # Optimize the model's selected variables


        ### END CODE HERE ###

    return total_loss

**Run the Training Loop**

In [ ]:
print('Start fine-tuning!', flush=True)

for idx in range(num_batches):
    # Grab keys for a random subset of examples
    all_keys = list(range(len(train_images_np)))
    random.shuffle(all_keys)
    example_keys = all_keys[:batch_size]

    # Get the ground truth
    gt_boxes_list = [gt_box_tensors[key] for key in example_keys]
    gt_classes_list = [gt_classes_one_hot_tensors[key] for key in example_keys]

    # get the images
    image_tensors = [train_image_tensors[key] for key in example_keys]

    # Training step (forward pass + backwards pass)
    total_loss = train_step_fn(image_tensors,
                               gt_boxes_list,
                               gt_classes_list,
                               detection_model,
                               optimizer,
                               to_fine_tune
                              )

    if idx % 10 == 0:
        print('batch ' + str(idx) + ' of ' + str(num_batches)
        + ', loss=' +  str(total_loss.numpy()), flush=True)

print('Done fine-tuning!')

Start fine-tuning!
batch 0 of 100, loss=1.5091523
batch 10 of 100, loss=1.6050851
batch 20 of 100, loss=1.3391234
batch 30 of 100, loss=0.82051253
batch 40 of 100, loss=0.608968
batch 50 of 100, loss=0.5170442
batch 60 of 100, loss=0.43395132
batch 70 of 100, loss=0.38090706
batch 80 of 100, loss=0.3664794
batch 90 of 100, loss=0.3494301
Done fine-tuning!


**Preprocess, predict, and post process an image**

In [ ]:
# Again, uncomment this decorator if you want to run inference eagerly
@tf.function
def detect(input_tensor):
    """Run detection on an input image.

    Args:
    input_tensor: A [1, height, width, 3] Tensor of type tf.float32.
      Note that height and width can be anything since the image will be
      immediately resized according to the needs of the model within this
      function.

    Returns:
    A dict containing 3 Tensors (`detection_boxes`, `detection_classes`,
      and `detection_scores`).
    """
    preprocessed_image, shapes = detection_model.preprocess(input_tensor)
    prediction_dict = detection_model.predict(preprocessed_image, shapes)

    ### START CODE HERE (Replace instances of `None` with your code) ###
    # use the detection model's postprocess() method to get the the final detections
    detections = detection_model.postprocess(prediction_dict, shapes)

    ### END CODE HERE ###

    return detections

In [ ]:
import cv2
import pandas as pd

test_image_dir = '/content/drive/MyDrive/Database/testing_calcium_assay_jpg/videos'
items = os.listdir(test_image_dir)
for x in range(len(items)):
    test_video_path = os.path.join(test_image_dir+'/'+items[x])
    test_images_np = []
    vid = cv2.VideoCapture(test_video_path)
    for i in range (1000):
        ret, frame = vid.read() # ret means whether there is a frame or not, frame is nd.array 968x548x3
        test_images_np.append(np.expand_dims(frame, axis=0))

    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()

    ##############################################################################
    # Note that the first frame will trigger tracing of the tf.function, which will
    # take some time, after which inference should be fast.

    label_id_offset = 1
    box_infos = {'x':[], 'y':[], 'mean_brightness':[],'frame':[],'score':[],'boxes':[]}

    for j in range(len(test_images_np)):
    # for j in range(0,999,100):
        this_frame = test_images_np[j][0,:,:,0]
        input_tensor = tf.convert_to_tensor(test_images_np[j], dtype=tf.float32)
        detections = detect(input_tensor) # detections in a single frame
        # filtered_detections = detections[detections['detection_scores'].numpy()>0.3]
        for i in range(len(detections['detection_boxes'][0])): # number of boxes in this frame
            if detections['detection_scores'][0][i].numpy() > 0.1:
                this_box = detections['detection_boxes'][0][i].numpy() # (y_min,x_min,y_max,x_max)
                if (this_box[2]-this_box[0])*(this_box[3]-this_box[1])<0.1: # max area: 0.1 x whole frame
                    x_center = int((this_box[1]+this_box[3])/2*968)
                    y_center = int((this_box[0]+this_box[2])/2*548)
                    box_infos['x'].append(x_center)
                    box_infos['y'].append(y_center)
                    box_infos['mean_brightness'].append(np.mean(this_frame[y_center-3:y_center+3,x_center-3:x_center+3]))
                    box_infos['frame'].append(j)
                    box_infos['score'].append(detections['detection_scores'][0][i].numpy())
                    box_infos['boxes'].append(this_box)

    box_infos_df = pd.DataFrame.from_dict(box_infos)
    test_video_results_path = os.path.join('/content/drive/MyDrive/Database/testing_calcium_assay_jpg/video_results/'+items[x] + '.csv')
    box_infos_df.to_csv(test_video_results_path)
    #######################################################
    # Note that the first frame will trigger tracing of the tf.function, which will
    # take some time, after which inference should be fast.
    label_id_offset = 1
    results = {'boxes': [], 'scores': []}

    for i in range(0,999,100):
    # for i in range(len(test_images_np)):
        input_tensor = tf.convert_to_tensor(test_images_np[i], dtype=tf.float32)
        detections = detect(input_tensor)
        plot_detections(
          test_images_np[i][0],
          detections['detection_boxes'][0].numpy(),
          detections['detection_classes'][0].numpy().astype(np.uint32)
          + label_id_offset,
          detections['detection_scores'][0].numpy(),
          category_index, figsize=(15, 20), image_name="/content/drive/MyDrive/Database/testing_calcium_assay_jpg/results/" + items[x] + ('%03d' % i) + ".jpg")
        results['boxes'].append(detections['detection_boxes'][0][0].numpy())
        results['scores'].append(detections['detection_scores'][0][0].numpy())